In [2]:
%matplotlib inline

import re
import pandas as pd
import os
import seaborn as sns



In [3]:
df = pd.read_csv('data/dd_data_all.csv')

In [4]:
df.drop(df[pd.isnull(df['sentences_1000_str'])].index, inplace=True)

In [5]:
df.drop(df[df['content'].str.len()<30].index, inplace=True)

In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np

Using TensorFlow backend.


In [7]:
MAX_SEQUENCE_LENGTH = 1000
EMBEDDING_DIM = 200


tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['sentences_1000_str'])
sequences = tokenizer.texts_to_sequences(df['sentences_1000_str'])


In [8]:
word_index = tokenizer.word_index

In [9]:
all_data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(df['class_no']))
print('Shape of data tensor:', all_data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (96393, 1000)
Shape of label tensor: (96393, 14)


In [10]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val = train_test_split(all_data,labels,test_size=0.3,stratify=labels)

In [11]:
x_train.shape,y_train.shape

((67475, 1000), (67475, 14))

In [15]:
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding, GlobalMaxPooling1D
from keras.models import Sequential

model = Sequential()
model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(Dropout(0.2))
model.add(Conv1D(250, 3, padding='valid', activation='relu', strides=1))
model.add(MaxPooling1D(3))
model.add(Flatten())
model.add(Dense(EMBEDDING_DIM, activation='relu'))
model.add(Dense(labels.shape[1], activation='sigmoid'))
model.summary()

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop', metrics=['accuracy'])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1000, 200)         139369800 
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000, 200)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 998, 250)          150250    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 332, 250)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 83000)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 200)               16600200  
_________________________________________________________________
dense_4 (Dense)              (None, 14)                2814      
Total para

In [16]:
model.fit(x_train, y_train, batch_size=128, epochs=5, verbose=1, validation_data=(x_val, y_val))  # starts training


/opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:105: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 139369800 elements. This may consume a large amount of memory.
  num_elements)
/opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:105: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 139369800 elements. This may consume a large amount of memory.
  num_elements)


Train on 67475 samples, validate on 28918 samples
Epoch 1/5
67475/67475 [==============================] - 57s 848us/step - loss: 0.1182 - acc: 0.9558 - val_loss: 0.0868 - val_acc: 0.9638
Epoch 2/5
67475/67475 [==============================] - 53s 790us/step - loss: 0.0701 - acc: 0.9695 - val_loss: 0.0860 - val_acc: 0.9628
Epoch 3/5
67475/67475 [==============================] - 53s 790us/step - loss: 0.0566 - acc: 0.9745 - val_loss: 0.0916 - val_acc: 0.9602
Epoch 4/5
67475/67475 [==============================] - 53s 791us/step - loss: 0.0475 - acc: 0.9777 - val_loss: 0.0998 - val_acc: 0.9593
Epoch 5/5
67475/67475 [==============================] - 53s 788us/step - loss: 0.0425 - acc: 0.9796 - val_loss: 0.1079 - val_acc: 0.9574


In [17]:
predictions = model.predict(pad_sequences(tokenizer.texts_to_sequences([df['sentences_1000_str'][1],]),MAX_SEQUENCE_LENGTH))


In [19]:
predictions[predictions>=0.5] = 1


In [21]:
predictions[predictions<0.5] = 0


In [22]:
predictions

array([[1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)

In [23]:
df['sentences_1000_str'][1]

'戴德 梁行 董事 總經理 顏炳立 認為 年 房市 落底 估計 年 落底 後 再 盤整 年 整體 走勢 如同 碟子 型 年 逐步 回彈 近期 北 市 商辦 市場 成為 資金 標的 台灣 電力公司 今天 舉行 北部 儲運 中心 南港 舊址 南側 土地 更案 招商 說明會 戴德 梁行 邀集 相關 廠商 舉行 開發 座談會 顏炳立 負責 分析 房市 走勢 顏炳立 表示 年 房市 斷 買單 風 一年 年 房市 落底 前 坡 相當 陡 利 再 價再降 量 再 縮 一年 預售 市場 量 再 縮 價再 破 北 市 蛋黃 區 房價 不黃 蛋白 區則 變黑 顏炳立 指出 年 房市 買賣 移轉 棟 數只 萬棟 年 回到 萬棟 看似 量 增 認為 房市 未 真正 落底 估計 年 落底 底部 盤整 年 年 再 如同 碟子 形狀 仍要 價格 買氣 成交量 相互 搭配 顏炳立 認為 年 商用 級 辦公大樓 缺貨 空置率 下降 引發 租金 房價 地價 上揚 優質 土地 難尋 以台電 釋出 南港區 坪 土地 準備 興建 住宅 商辦 滿足 部分 市場 資金 去處 主要 南港區 北 市府 東區 門戶 計畫 重要 大門 南港區 商用 土地 蓄勢待發 可望 成為 資金 追逐 標的 建議 最近 地主 讓利 降價 土地 市場 買賣 加溫 跡象 建商 努力 標購 存地 建商 應 慎用 子彈 備糧 風 戴德 梁行 董事 總經理 顏炳立 認為 年 房市 落底 估計 年 落底 後 再 盤整 年 整體 走勢 如同 碟子 型 年 逐步 回彈 近期 北 市 商辦 市場 成為 資金 標的 聯合報系 資料 記者 毛洪霖 攝影 文章 來源 udn 文章 標籤 房市 房市 落底 創作者 介紹 amy010203 房產 明星'

In [42]:
predictions = model.predict(pad_sequences(tokenizer.texts_to_sequences([df['sentences_1000_str'][60000],]),MAX_SEQUENCE_LENGTH))
predictions[predictions>=0.5] = 1
predictions[predictions<0.5] = 0


In [43]:
predictions

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)

In [44]:
df['sentences_1000_str'][60000]

'分享 分享 留言 列印 聯合 新聞網 Lucas LINE 傳送 關鍵字 FordGT ‧ Ford ‧ 跑車 擁有 一輛 FordGT rel gt FordGT 超級 跑車 熱血 車迷 福特 鐵粉 夢想 每年 輛 生產量 人 有錢 買不到 而本 週末 將會 有個 入主 FordGT 機會 先 準備 一大筆錢 飛去 美國 競標 MecumAuctions 提供 facebook 搭載 銀色 車身 烤 漆 車頭 延伸 車尾 黑色 飾條 這輛 年式 FordGT 超級 跑車 去年 出廠 輛中 輛 車款 配置 高光澤 碳纖維 套件 銀色 鍛造 輪圈 含 鈦合金 輪圈 螺帽 銀色 煞車 卡鉗 黑色 座艙 車輛 總行 駛 里程 英里 約 公里 上路 確實 買家 心動 MecumAuctions 提供 facebook 美國 研發 並在 加拿大 生產 FordGT 超級 跑車 搭載 爆發 匹 最大 馬力 公斤 米巔 峰值 扭力 輸出 升 雙渦輪 增壓 V6 汽油 引擎 搭配 速雙 離合器 變速箱 系統 後輪 驅動 設定 小時 公里 加速 僅需 秒 極速 達到 小時 公里 MecumAuctions 提供 facebook 想 買 一輛 FordGT 超級 跑車 已經 容易 總 里程 個位數 字 更是 難上加難 接受 銀色 FordGT 下定決心 買 週末 快趕 美國 印地安納 州 首府 印第安納波利斯 參加 MecumAuctions 舉辦 拍賣會 出價 最高者 能將 FordGT 開 回家 MecumAuctions 提供 facebookMecumAuctions 提供 facebookMecumAuctions 提供 facebookMecumAuctions 提供 facebook'